In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import cProfile
from rolling_lookahead_dt_pulp import rollo_oct_pulp
import os
import time

import pickle

In [ ]:
if __name__ == "__main__":
    # Parameter aus dem Aufrug von run
    depth_rolling_tree = 8
    time_limit = 1800
    criterion_loss = "gini"
    #amount_cores = 7
    #big_dataset = True #True ca. über 100 bin features

    # test, car_evaluation, seismic, wine, adult (ordered by amount bin features)
    dataset_name = 'test' # folder in 'results' will be named after it, results/dataset_name contains result .txt and .csv

    #anpassen je nachdem was gerade getestet wird
    dir_path = f'results/{dataset_name}/not_cval/old_gini'
    file_path = f'{dir_path}/output_depth_'

    # Create the directory if it doesn't exist
    os.makedirs(dir_path, exist_ok=True)

    if dataset_name == 'test':
    # Load your training and test datasets
        data = pd.read_csv("datasets/example_datasets/stacked.csv")

    if dataset_name == 'car_evaluation':
    # Load your training and test datasets
        data = pd.read_csv("datasets/car_evaluation/car_bin.csv")

    if dataset_name == 'seismic':
    # Load your training and test datasets
        data = pd.read_csv("datasets/seismic/seismic_bin.csv")

    if dataset_name == 'wine':
    # Load your training and test datasets
        data = pd.read_csv("datasets/wine/wine_bin.csv")

    if dataset_name == 'adult':
    # Load your training and test datasets
        data = pd.read_csv("datasets/adult/stacked.csv")

    X = data.drop(columns=['y'])  # All columns except the target
    y = data['y']                 # Only the target column

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=100, stratify=y, random_state=42)

    stacked_train = pd.concat([y_train, X_train], axis=1, ignore_index=False)
    stacked_test = pd.concat([y_test, X_test],axis=1, ignore_index=False)

    train_data = stacked_train
    test_data = stacked_test

    feature_columns = train_data.columns[1:]

    start_time_pulp = time.time()

    #profiler = cProfile.Profile()
    #profiler.enable()
    result_dict_pulp =rollo_oct_pulp.run(train=train_data, test=test_data, target_label="y", features=feature_columns, depth=depth_rolling_tree, criterion=criterion_loss)
    #profiler.disable()
    #profiler.dump_stats("rollOCT_gini_multiprocess.out")

    end_time_pulp = time.time()

    for depth in range(2,depth_rolling_tree+1):
        print(result_dict_pulp[depth])

    print(f"Pulp execution time for depth {depth_rolling_tree} : {end_time_pulp - start_time_pulp} seconds\n")
    """
    for depth in range(2,depth_rolling_tree+1):
        with open(f'{file_path}all_{dataset_name}.txt', 'w') as f:
            f.write(str(depth) + ': ' + str(result_dict_pulp[depth]) + "\n")

    with open(f'{file_path}all_{dataset_name}.txt', 'a') as f:
            f.write(f"Pulp execution time for depth {depth_rolling_tree} : {end_time_pulp - start_time_pulp} seconds\n")

    for depth in range(2,depth_rolling_tree+1):
        with open(f'{file_path}{depth}_classification_{dataset_name}_test_pulp.csv', 'w') as f:
            f.write(str(result_dict_pulp['tree'][depth]['test'].to_csv()))
        with open(f'{file_path}{depth}_classification_{dataset_name}_train_pulp.csv', 'w') as f:
            f.write(str(result_dict_pulp['tree'][depth]['train'].to_csv()))
    """
    
    os.makedirs(f'predict_model_dicts/testruns/{dataset_name}', exist_ok=True)

    for depth in range(2, depth_rolling_tree+1):
        with open(f'predict_model_dicts/testruns/{dataset_name}/predict_dict_{depth}.pkl', 'wb') as f: # 'wb' for write binary, rb for read binary
            pickle.dump(result_dict_pulp['tree'][depth]['trained_dict'], f)
    


{'leaf_nodes': [4, 5, 6, 7], 'leaf_nodes_path': {4: [1, 1], 5: [1, 0], 6: [0, 1], 7: [0, 0]}}
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/drood/Obsidian/Files/Bachelorarbeit/rlrf_my_try/.venv/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/129d583b79af4592a15dde3d9aa354c1-pulp.mps -sec 1800 -timeMode elapsed -branch -printingOptions all -solution /tmp/129d583b79af4592a15dde3d9aa354c1-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 137 COLUMNS
At line 167877 RHS
At line 168010 BOUNDS
At line 201811 ENDATA
Problem MODEL has 132 rows, 33800 columns and 67600 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 1800
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0.408321 - 0.02 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to d